# Projet maison n° 3

In [1]:
# imports
import pandas as pd

import numpy as np

import unittest

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us

def df_names_us():
    
    #year = [1880, ... ,1881]
    years = np.arange(1880, 2021, 1)

    path = "names"

    #files = [path+'/yob1880.txt', path+'/yob1881.txt, ..., path+'/yob2020.txt']
    files = []
    
    #On parcourt le dossier contenant les fichiers txt
    for year in years:
        file = path+'/yob'+str(year)+'.txt'
        files.append(file)

    frames = []

    # On parcourt la liste de fichiers, on lit le fichier et on transforme en Dataframe, on ajoute ce dataframe à la liste de dataframes
    for file in files:
        frames.append(pd.read_csv(file, sep = ",", header=None))

    # on parcourt la liste de dataframe pour renommer les colonnes de chaque dataframe et on y ajoute la colonne year    
    for i in range (0, len(frames), 1):
        frames[i] = frames[i].rename(columns={0: 'name', 1: 'gender', 2: 'births'})
        frames[i]["year"] = years[i]

    # On concatène chaque dataframe contenu dans la liste frame pour créer un dataframe unique
    df = pd.concat(frames, axis=0)

    # On réorganise les colonnes des datasets : La colonne 'year' passe on première position
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    # On modifie le type des index
    df.index = pd.RangeIndex(start=0, stop=2020863, step=1)
    
    return df

df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2020.csv", sep = ";")

    df.head()

    # On renomme les colonnes 
    df = df.rename(columns={'sexe': 'gender', 'preusuel': 'name', 'annais': 'year', 'nombre': 'births'})

    # On réorganise l'ordre des colonnes pour avoir 'year', 'name', 'gender', 'birth'
    df = df.reindex(columns=['year', 'name', 'gender', 'births'])
    #df.head()

    # On s'occupe du typage des colonnes
    df = df.astype(dict(zip(['year', 'name', 'gender', 'births'], [np.int32, str, str, np.int32])), errors='ignore')

    # On s'occupe des valeur de la colonne 'gender' Si gender=1 alors on remplace par M sinon on remplace par F
    df.loc[df['gender'] == "1", 'gender'] = 'M'
    df.loc[df['gender'] == "2", 'gender'] = 'F'

    #print(df.head())

    # On ne garde que les noms qui supérieures ou égales à 2 caractères
    df = df[df['name'].str.len() >= 2]

    # On regarde si il exsite des valeurs manquantes dans le dataset (NaN) pour chaque colonne
    df.isnull().sum(axis = 0)

    # Output : Pas de valeurs manquantes dans le dataset sur chaque colonne 

    # On supprime les lignes contenant des valeurs incompréhensible
    # En parcourant le dataset on remarque les patterns "XXXX" pour la colonne 'year' et "_PRENOMS_RARES" pour la colonne 'name'
    # On supprime ces 2 patterns

    df = df[df['name'] != "_PRENOMS_RARES"]
    df = df[df['year'] != "XXXX"]

    #df.head()

    # On s'occupe maintenant du type de format des valeurs de la colonnes 'name' tous les mots doivent commencer par une majuscule

    df['name'] = df['name'].str.title()

    #df.head()

    # On ré-oragnise les lignes du dataset de la manière suivante : 
    # D'abord par année croissante puis par sexe (sexe féminin en premier) puis par ordre décroissant le nombre d'occurence du nom (colonne births) puis les noms par ordre alaphébitque 
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True], ignore_index=True)

    return df

df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630402,2020,Élouen,M,3
630403,2020,Énes,M,3
630404,2020,Éphraïm,M,3
630405,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [4]:
# taux de change
def df_taux_change(devises):
    
    # On lit le fichier CSV, on ne prend pad en compte les 5 premières lignes du dataset car elle ne nous interesse pas dans le cadre de notre étude
    df = df=pd.read_csv('Webstat_Export_20210929.csv', sep=";",skiprows=[1,2,3,4,5], index_col='Titre :')

    #df.head()
    
    # On remarque des valeurs imcompréhensibles du type "-" et "" (string vide)
    # On supprime ces valeurs 

    df = df[df[:]!= "-"]
    df = df[df[:]!= ""]


    # print(df)

    # On supprime les valeurs manquantes sur tous les datasets
    df = df.dropna()

    #df

    # On converti la colonne titre qui contient des dates en format DateTime
    df.index = pd.to_datetime(df.index,format='%d/%m/%Y')

    # On renomme les colonnes du dataset pour ne garder que les devises contenu dans les parenthèse 
    # Exemple : Dollar australien (AUD) => AUD
    list_devise = []

    for colonne in df.columns:
        devise = colonne.split("(")
        devise = devise[1].split(")")
        list_devise.append(devise[0])

    #print(list_devise)

    df.columns = list_devise

    #df

    # On renomme l'index du dataframe par "Date"
    df=df.rename_axis('Date')

    # On transforme les virgules des valeurs des devises par des points car python interprête un flottant par un '.'
    # On converti chaque valeur de devises en nombre flottant
    df=df.replace(',','.',regex=True).astype(float)

    df = pd.DataFrame(df[devises], columns=devises)
    
    return df

df_taux_change(['AUD','CYP'])

,AUD,CYP
Date,,
2006-12-29,1.6691,0.57820
2006-12-28,1.6699,0.57820
2006-12-27,1.6765,0.57820
2006-12-22,1.6792,0.57810
2006-12-21,1.6785,0.57810
...,...,...
2000-07-25,1.6072,0.57425
2000-07-24,1.5905,0.57395
2000-07-21,1.5945,0.57399


### Tests

In [5]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)  
        
    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.387s

OK
